#DATASCI W261: Machine Learning at Scale 

* **Sayantan Satpati**
* **sayantan.satpati@ischool.berkeley.edu**
* **W261**
* **Week-5**
* **Assignment-5**
* **Date of Submission: 07-OCT-2015**

#  === Week 5: mrjob, aws, and n-grams ===

## HW 5.0
---

***What is a data warehouse? What is a Star schema? When is it used?***


## HW 5.1
---

***In the database world What is 3NF? Does machine learning use data in 3NF? If so why? ***

***In what form does ML consume data?***

***Why would one use log files that are denormalized?***

## HW 5.2
---

Using MRJob, implement a hashside join (memory-backed map-side) for left, 
right and inner joins. Run your code on the  data used in HW 4.4: (Recall HW 4.4: Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.)
:

Justify which table you chose as the Left table in this hashside join.

Please report the number of rows resulting from:

```
(1) Left joining Table Left with Table Right
(2) Right joining Table Left with Table Right
(3) Inner joining Table Left with Table Right
```

## HW 5.3
---

For the remainder of this assignment you will work with a large subset 
of the Google n-grams dataset,

https://aws.amazon.com/datasets/google-books-ngrams/

which we have placed in a bucket on s3:

s3://filtered-5grams/

In particular, this bucket contains (~200) files in the format:

(ngram) \t (count) \t (pages_count) \t (books_count)

Do some EDA on this dataset using mrjob, e.g., 

- Longest 5-gram (number of characters)
- Top 10 most frequent words (count), i.e., unigrams
- Most/Least densely appearing words (count/pages_count) sorted in decreasing order of relative frequency (Hint: save to PART-000* and take the head -n 1000)
- Distribution of 5-gram sizes (counts) sorted in decreasing order of relative frequency. (Hint: save to PART-000* and take the head -n 1000)
OPTIONAL Question:
- Plot the log-log plot of the frequency distributuion of unigrams. Does it follow power law distribution?

For more background see:
https://en.wikipedia.org/wiki/Log%E2%80%93log_plot
https://en.wikipedia.org/wiki/Power_law

### Longest 5-gram (number of characters)

### Part (A)
---

In [12]:
%%writefile mrjob_hw53_1.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re


class LongestNgram(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_ngrams_len,
                   reducer=self.reducer_ngrams_len),
            MRStep(reducer=self.reducer_find_max_ngram)
        ]

    def mapper_ngrams_len(self, _, line):
        tokens = line.strip().split('\t')
        yield (tokens[0], len(tokens[0]))

  
    def reducer_ngrams_len(self, word, counts):
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_ngram(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)


if __name__ == '__main__':
    LongestNgram.run()

Overwriting mrjob_hw53_1.py


In [13]:
!chmod a+x mrjob_hw53_1.py

In [16]:
%load_ext autoreload
%autoreload 2
from mrjob_hw53_1 import LongestNgram
mr_job = LongestNgram(args=['s3://filtered-5grams',
                            '-r', 'emr', '--no-strict-protocol'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(159, 'ROPLEZIMPREDASTRODONBRASLPKLSON YHROACLMPARCHEYXMMIOUDAVESAURUS PIOFPILOCOWERSURUASOGETSESNEGCP TYRAVOPSIFENGOQUAPIALLOBOSKENUO OWINFUYAIOKENECKSASXHYILPOYNUAT')


### Part (B)
---

In [26]:
%%writefile mrjob_hw53_2.py
#!/usr/bin/python
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
from itertools import combinations

class FrequentUnigrams(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer),
             MRStep(mapper=self.mapper_frequent_unigrams,
                   reducer=self.reducer_frequent_unigrams,
                   jobconf={
                            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                            'mapred.text.key.comparator.options': '-k1,1rn',
                            }
                   )
        ]
    
    def mapper(self, _, line):
        line.strip()
        tokens = re.split("\t",line)
        unigrams = tokens[0].split()
        count = int(tokens[1])
        for unigram in unigrams:
            yield unigram, count
    
    def combiner(self, unigram, counts):
        yield unigram, sum(counts)
    
    def reducer(self, unigram, counts):
        yield unigram, sum(counts)
        
    def mapper_frequent_unigrams(self, unigram, count):
        yield count, unigram
        
    def reducer_frequent_unigrams(self, count, unigrams):
        for unigram in unigrams:
            yield count, unigram
    
if __name__ == '__main__':
    FrequentUnigrams.run()

Overwriting mrjob_hw53_2.py


In [27]:
!chmod a+x mrjob_hw53_2.py

In [31]:
!python mrjob_hw53_2.py -r emr \
 s3://filtered-5grams \
 --output-dir=s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words \
 --no-output \
 --no-strict-protocol

using configs in /Users/ssatpati/.mrjob.conf
using existing scratch bucket mrjob-d5ed1dc31babbe2c
using s3://mrjob-d5ed1dc31babbe2c/tmp/ as our scratch dir on S3
creating tmp directory /var/folders/h5/1q71m1c54cn07f16c232pqgm38ynd8/T/mrjob_hw53_2.ssatpati.20151003.062810.595234
writing master bootstrap script to /var/folders/h5/1q71m1c54cn07f16c232pqgm38ynd8/T/mrjob_hw53_2.ssatpati.20151003.062810.595234/b.py
Copying non-input files into s3://mrjob-d5ed1dc31babbe2c/tmp/mrjob_hw53_2.ssatpati.20151003.062810.595234/files/
Waiting 5.0s for S3 eventual consistency
Creating Elastic MapReduce job flow
Job flow created with ID: j-QGLC3PJPOLBE
Created new job flow j-QGLC3PJPOLBE
Job launched 30.4s ago, status STARTING: Provisioning Amazon EC2 capacity
Job launched 61.3s ago, status STARTING: Provisioning Amazon EC2 capacity
Job launched 91.8s ago, status STARTING: Provisioning Amazon EC2 capacity
Job launched 122.8s ago, status STARTING: Provisioning Amazon EC2 capacity
Job launched 153.1s ago

In [41]:
!rm -rf ./output;mkdir output;cd output;aws s3 cp --recursive s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words .;
!cat output/part-0000* | sort -nrk 1 | head -n 10
!rm -rf ./output

download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/_SUCCESS to ./_SUCCESS
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00003 to ./part-00003
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00000 to ./part-00000
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00002 to ./part-00002
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00004 to ./part-00004
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00006 to ./part-00006
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00001 to ./part-00001
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00005 to ./part-00005
5375699242	"the"
3691308874	"of"
2221164346	"to"
1387638591	"in"
1342195425	"a"
1135779433	"and"
798553959	"that"
756296656	"is"
688053106	"be"
481373389	"as"
sort: write failed: standard output: Broken pipe
sort: write error


### Top 10 Frequent Words are:

```
5375699242	"the"
3691308874	"of"
2221164346	"to"
1387638591	"in"
1342195425	"a"
1135779433	"and"
798553959	"that"
756296656	"is"
688053106	"be"
481373389	"as"
```

### Part (C)
---

In [34]:
%%writefile mrjob_hw53_3.py
#!/usr/bin/python
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
from itertools import combinations

class DenseWords(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer,
                   ),
            MRStep(mapper=self.mapper_max_min,
                   reducer=self.reducer_max_min,
                   jobconf={
                            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                            'mapred.text.key.comparator.options': '-k1,1rn',
                            }
                   )
        ]
    
    def mapper(self, _, line):
        tokens = line.strip().split('\t')
        unigrams = tokens[0].split()
        density = round((int(tokens[1]) * 1.0 / int(tokens[2])), 3)
        for unigram in unigrams:
            yield unigram, density
            
    def combiner(self, unigram, densities):
        densities = [d for d in densities]
        yield unigram, min(densities) 
        yield unigram, max(densities)
        
    def reducer(self, unigram, densities):
        densities = [d for d in densities]
        yield unigram, min(densities)
        yield unigram, max(densities)
        
    def mapper_max_min(self, unigram, density):
        yield density, unigram
        
    def reducer_max_min(self, density, unigrams):
        for unigram in unigrams:
            yield density, unigram
    
if __name__ == '__main__':
    DenseWords.run()

Overwriting mrjob_hw53_3.py


In [35]:
!chmod a+x mrjob_hw53_3.py

In [36]:
!aws s3 rm --recursive s3://ucb-mids-mls-sayantan-satpati/hw53/density/
!python mrjob_hw53_3.py -q -r emr \
 s3://filtered-5grams \
 --output-dir=s3://ucb-mids-mls-sayantan-satpati/hw53/density \
 --no-output \
 --no-strict-protocol

delete: s3://ucb-mids-mls-sayantan-satpati/hw53/density/_SUCCESS
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00001
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00000
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00002
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00004
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00003
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00005
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00006


In [96]:
!rm -rf ./output;mkdir output;cd output;aws s3 cp --recursive s3://ucb-mids-mls-sayantan-satpati/hw53/density/ .;
!cat output/part-0000* | sort -nrk 1 > merged
!head -n 10 merged
!tail -n 10 merged
!rm -rf ./output

download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/_SUCCESS to ./_SUCCESS
download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00003 to ./part-00003
download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00004 to ./part-00004
download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00005 to ./part-00005
download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00006 to ./part-00006
download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00000 to ./part-00000
download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00001 to ./part-00001
download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00007 to ./part-00007
download: s3://ucb-mids-mls-sayantan-satpati/hw53/density/part-00002 to ./part-00002
29.704999999999998	"Death"
14.912000000000001	"write"
14.912000000000001	"and"
12.380000000000001	"NA"
11.557	"xxxx"
11.557	"xxxx"
10.882	"Sc"
10.6	"beep"
10.022	"blah"
9.8040000000000003	"whole"
1.0	"AAAS"
1.0	"AAAI"
1.0	"AAAE"
1.0	"AAAE"

### Top 10 Most Dense Words

```
29.704999999999998	"Death"
14.912000000000001	"write"
14.912000000000001	"and"
12.380000000000001	"NA"
11.557	"xxxx"
11.557	"xxxx"
10.882	"Sc"
10.6	"beep"
10.022	"blah"
9.8040000000000003	"whole"
```

### Top 10 Least Dense Words

```
1.0	"AAAS"
1.0	"AAAI"
1.0	"AAAE"
1.0	"AAAE"
1.0	"AAAA"
1.0	"AAAA"
1.0	"AAA"
1.0	"AA"
1.0	"A's"
1.0	"A"
```


### Part (D)
---

In [40]:
%%writefile mrjob_hw53_4.py
#!/usr/bin/python
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
from itertools import combinations

class DistributionNgram(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   reducer=self.reducer,
                   jobconf={
                            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                            'mapred.text.key.comparator.options': '-k1,1rn',
                            }
                   )
        ]
    
    def mapper(self, _, line):
        line.strip()
        tokens = re.split("\t",line)
        yield int(tokens[1]), tokens[0]
    
    def reducer(self, count, ngrams):
        for ngram in ngrams:
            yield count, ngram
    
if __name__ == '__main__':
    DistributionNgram.run()

Overwriting mrjob_hw53_4.py


In [41]:
!chmod a+x mrjob_hw53_4.py

In [16]:
!aws s3 rm --recursive s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/
!python mrjob_hw53_4.py -q -r emr \
 s3://filtered-5grams/ \
 --output-dir=s3://ucb-mids-mls-sayantan-satpati/hw53/distribution \
 --no-output \
 --no-strict-protocol

delete: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/_SUCCESS
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00001
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00006
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00000
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00003
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00002
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00004
delete: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00005


In [19]:
!rm -rf ./output;mkdir output;cd output;aws s3 cp --recursive s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/ .;
!cat output/part-0000* | sort -nrk 1 > output_hw53_4.txt
!rm -rf ./output
!head -n 10 output_hw53_4.txt

download: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/_SUCCESS to ./_SUCCESS
download: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00000 to ./part-00000
download: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00002 to ./part-00002
download: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00001 to ./part-00001
download: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00004 to ./part-00004
download: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00003 to ./part-00003
download: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00005 to ./part-00005
download: s3://ucb-mids-mls-sayantan-satpati/hw53/distribution/part-00006 to ./part-00006
sort: write failed: standard output: Broken pipe
sort: write error
cat: stdout: Broken pipe
2223343	"on the basis of the"
1462063	"at the head of the"
1316960	"as well as in the"
871158	"at the same time the"
833764	"as a matter of fact"
590885	"as a part of the"
544783	"as one of th

### Top 10 NGram Distribution

```
2223343	"on the basis of the"
1462063	"at the head of the"
1316960	"as well as in the"
871158	"at the same time the"
833764	"as a matter of fact"
590885	"as a part of the"
544783	"as one of the most"
533464	"and at the end of"
447364	"I do not think that"
441568	"from the beginning of the"
```

## HW 5.3
---

In this part of the assignment we will focus on developing methods
for detecting synonyms, using the Google 5-grams dataset. To accomplish
this you must script two main tasks using MRJob:

(1) Build stripes of word co-ocurrence for the top 10,000
most frequently appearing words across the entire set of 5-grams,
and output to a file in your bucket on s3 (bigram analysis, though the words are non-contiguous).

(2) Using two (symmetric) comparison methods of your choice 
(e.g., correlations, distances, similarities), pairwise compare 
all stripes (vectors), and output to a file in your bucket on s3.

==Design notes for (1)==
For this task you will be able to modify the pattern we used in HW 3.2
(feel free to use the solution as reference). To total the word counts 
across the 5-grams, output the support from the mappers using the total 
order inversion pattern:

<*word,count>

to ensure that the support arrives before the cooccurrences.

In addition to ensuring the determination of the total word counts,
the mapper must also output co-occurrence counts for the pairs of
words inside of each 5-gram. Treat these words as a basket,
as we have in HW 3, but count all stripes or pairs in both orders,
i.e., count both orderings: (word1,word2), and (word2,word1), to preserve
symmetry in our output for (2).

==Design notes for (2)==
For this task you will have to determine a method of comparison.
Here are a few that you might consider:

- Spearman correlation
- Euclidean distance
- Taxicab (Manhattan) distance
- Shortest path graph distance (a graph, because our data is symmetric!)
- Pearson correlation
- Cosine similarity
- Kendall correlation
...

### HW 5.3: Part 1 > Step 1:

***Find the frequent unigrams for the top 10,000 most frequently appearing words across the entire set of 5-grams***

* Use the freqency count output from earlier step to get the top 10K most frequent words

In [109]:
!rm -rf ./output;mkdir output;cd output;aws s3 cp --recursive s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words .;
!cat output/part-0000* | sort -nrk 1 | head -n 10000 > output/frequent_unigrams.txt
!head -n 10 output/frequent_unigrams.txt

download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/_SUCCESS to ./_SUCCESS
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00006 to ./part-00006
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00002 to ./part-00002
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00000 to ./part-00000
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00005 to ./part-00005
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00003 to ./part-00003
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00004 to ./part-00004
download: s3://ucb-mids-mls-sayantan-satpati/hw53/frequent_words/part-00001 to ./part-00001
sort: write failed: standard output: Broken pipe
sort: write error
5375699242	"the"
3691308874	"of"
2221164346	"to"
1387638591	"in"
1342195425	"a"
1135779433	"and"
798553959	"that"
756296656	"is"
688053106	"be"
481373389	"as"


### HW 5.3: Part 1 > Step 2:

***Build stripes of word co-ocurrence for the top 10,000
most frequently appearing words across the entire set of 5-grams***

* Pass the 1 item frequent set generated from earlier step to the mapper(s) for filtering
* The reducers would sum and emit Word Co-Occurrence if support count of 10,000 is met

In [117]:
%%writefile word_cooccurrences_set_hw54.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import sys


class WordCoOccurrenceFrequentSet(MRJob):

    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer)
        ]
    
    def mapper_init(self):
        # Load the file into memory
        self.unigrams = {}
        with open('frequent_unigrams.txt', 'r') as f:
            for line in f:
                tokens = line.strip().split('\t')
                self.unigrams[tokens[1].replace("\"","")] = int(tokens[0])
        sys.stderr.write('### of unigrams: {0}'.format(self.unigrams))

    def mapper(self, _, line):
        tokens = line.strip().split('\t')
        # List of 5-grams
        words = tokens[0].split()
        # Filter 5-grams to only those in list
        words = [w for w in words if w in self.unigrams.keys()]
        l = len(words)
        for i in xrange(l):
            d = {}
            for j in xrange(l):
                if i != j:
                    d[words[j]] = d.get(words[j], 0) + 1
            # Emit word, stripe
            yield words[i],d

    def combiner(self, word, stripes):
        d = {}
        # Aggregate stripes
        for s in stripes:
            for k, v in s.iteritems():
                d[k] = d.get(k, 0) + v
        yield word,d
        
    def reducer(self, word, stripes):
        d = {}
        # Aggregate stripes
        for s in stripes:
            for k, v in s.iteritems():
                d[k] = d.get(k, 0) + v
        '''
        # Filter based on support count (Not a requirement!)
        d_final = {}
        for k,v in d.iteritems():
            if v >= 10000:
                d_final[k] = v
        '''
        # Combine stripes
        yield word,d

if __name__ == '__main__':
    WordCoOccurrenceFrequentSet.run()

Overwriting word_cooccurrences_set_hw54.py


In [118]:
!chmod a+x word_cooccurrences_set_hw54.py

In [ ]:
!aws s3 rm --recursive s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/
!python word_cooccurrences_set_hw54.py -q -r emr \
 s3://filtered-5grams/ \
 --file output/frequent_unigrams.txt \
 --output-dir=s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur \
 --no-output \
 --no-strict-protocol

#### HW 5.3 Part 2 > Step 1

***From the output of Part 1 > Step 2, create a single file with all the stripes - frequent itemsets of size 2 (s=10000)***

In [105]:
!rm -rf ./output;mkdir output;cd output;aws s3 cp --recursive s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/ .;
!cat output/part-0000* | sort -k 1 > output/frequent_stripes.txt
!wc -l output/frequent_stripes.txt
!head -n 10 output/frequent_stripes.txt

download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00000 to ./part-00000
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00006 to ./part-00006
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00002 to ./part-00002
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00007 to ./part-00007
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00004 to ./part-00004
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00005 to ./part-00005
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00009 to ./part-00009
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00003 to ./part-00003
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/_SUCCESS to ./_SUCCESS
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00010 to ./part-00010
download: s3://ucb-mids-mls-sayantan-satpati/hw54/word_cooccur/part-00008 to ./part-00008
download: s3:/

In [ ]:
%%writefile dist_calc_hw54.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re
import sys
import ast
from sets import Set
import math


class DistanceCalc(MRJob):

    def steps(self):
        return [
            MRStep(mapper_init=self.mapper_init,
                   mapper=self.mapper)
        ]
    
    def mapper_init(self):
        # Load the file into memory
        self.stripes = {}
        with open('frequent_stripes.txt', 'r') as f:
            for line in f:
                tokens = line.strip().split('\t')
                self.stripes[tokens[0].replace("\"","")] = ast.literal_eval(tokens[1])
        sys.stderr.write('### of stripes: {0}'.format(len(self.stripes)))

    def mapper(self, _, line):        
        tokens = line.strip().split('\t')
        key = tokens[0].replace("\"","")
        dict_pairs = ast.literal_eval(tokens[1])
        for n_key, n_dict_pairs in self.stripes.iteritems():
            # Do distance calc for only (a,b) but not (b,a) --> Redundant
            if key > n_key:
                continue
            
            s1 = Set(dict_pairs.keys())
            s2 = Set(n_dict_pairs.keys())
            common_keys = s1.intersection(s2)
            l_uniq = s1.difference(s2)
            r_uniq = s2.difference(s1)
            
            euclidean_distance = 0
            for k in common_keys:
                euclidean_distance += (dict_pairs.get(k) - n_dict_pairs.get(k)) ** 2
            for k in l_uniq:
                euclidean_distance += dict_pairs.get(k) ** 2
            for k in r_uniq:
                euclidean_distance += n_dict_pairs.get(k) ** 2
                
            yield (key, n_key, 'E'), math.sqrt(euclidean_distance)

if __name__ == '__main__':
    DistanceCalc.run()

In [ ]:
!chmod a+x dist_calc_hw54.py